In [2]:
################################################################################
# Teste do gridsearch com os dados utilizados no Experimento 2 (Elastic Tensor)
# (não funciona, com erro indicando problema com float)
# PARTE 1: PREPARAÇÃO DOS DADOS

# prepara o dataframe para os testes seguintes
# ao final temos um dataframe com os dados carregados e sem as colunas que não vamos estudar

from __future__ import print_function

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.kernel_ridge import KernelRidge


from matminer.datasets.convenience_loaders import load_elastic_tensor
df = load_elastic_tensor()  # loads dataset in a pandas DataFrame object
unwanted_columns = ["volume", "nsites", "compliance_tensor", "elastic_tensor", 
                    "elastic_tensor_original", "K_Voigt", "G_Voigt", "K_Reuss", "G_Reuss"]
df = df.drop(unwanted_columns, axis=1)

# aplicando featurizers

#coulomb matrix - flatten
# ao final teremos o dataframe "caracterizado", onde a coluna "estrutura" foi transformada em diversas colunas dos 
# eigenvalues da coulomb matrix

from matminer.featurizers.structure import CoulombMatrix

X = df['structure'].as_matrix()
cm = CoulombMatrix(flatten=True).fit(X)
df = cm.featurize_dataframe(df, "structure")


# separando os valores X e y_* apenas com as colunas da coulombx matrix
# e separando em train e val

from sklearn.model_selection import train_test_split
y_G_VRH = df['G_VRH'].values
y_K_VRH = df['K_VRH'].values
y_ea= df['elastic_anisotropy'].values

excluded = ["G_VRH", "K_VRH", "elastic_anisotropy", "formula", "material_id", 
            "poisson_ratio", "structure", "space_group"]
X = df.drop(excluded, axis=1)
#print("There are {} possible descriptors:\n\n{}".format(X.shape[1], X.columns.values))

train_X, val_X, train_y, val_y = train_test_split(X, y_G_VRH, random_state=1)

/home/luis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



/home/luis/anaconda3/lib/python3.6/site-packages/matminer/featurizers/structure.py:618: ComplexWarning:

Casting complex values to real discards the imaginary part

/home/luis/anaconda3/lib/python3.6/site-packages/matminer/featurizers/structure.py:618: ComplexWarning:

Casting complex values to real discards the imaginary part

/home/luis/anaconda3/lib/python3.6/site-packages/matminer/featurizers/structure.py:618: ComplexWarning:

Casting complex values to real discards the imaginary part

/home/luis/anaconda3/lib/python3.6/site-packages/matminer/featurizers/structure.py:618: ComplexWarning:

Casting complex values to real discards the imaginary part



In [7]:
################################################################################
# Teste do gridsearch com os dados utilizados no Experimento 2 (Elastic Tensor)
# PARTE 2: ANÁLISE


# Set the parameters by cross-validation
#"linear", "rbf", "poly", "cosine"
tuned_parameters = [{'kernel': ['linear'], 'alpha': [1, 2, 3, 5, 8, 13, 21, 34]},
                    {'kernel': ['rbf'], 'alpha': [1, 2, 3, 5, 8, 13, 21, 34]},
                    {'kernel': ['poly'], 'alpha': [1, 2, 3, 5, 8, 13, 21, 34]},
                    {'kernel': ['cosine'], 'alpha': [1, 2, 3, 5, 8, 13, 21, 34]},
                    ]


scores = ['r2', 'neg_mean_absolute_error',  'neg_mean_squared_error']


for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(KernelRidge(), tuned_parameters, cv=2,
                       scoring='%s' % score)
                       
    clf.fit(train_X, train_y)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    ############################################
    # Este bloco ainda está apresentando erro
    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = val_y, clf.predict(val_X)
    print(classification_report(y_true, y_pred))
    print()
    


# Tuning hyper-parameters for r2



/home/luis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:167: UserWarning:

Singular matrix in solving dual problem. Using least-squares solution instead.

/home/luis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:167: UserWarning:

Singular matrix in solving dual problem. Using least-squares solution instead.

/home/luis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:167: UserWarning:

Singular matrix in solving dual problem. Using least-squares solution instead.

/home/luis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:167: UserWarning:

Singular matrix in solving dual problem. Using least-squares solution instead.

/home/luis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:167: UserWarning:

Singular matrix in solving dual problem. Using least-squares solution instead.

/home/luis/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:167: UserWarning:

Singular mat

Best parameters set found on development set:

{'alpha': 8, 'kernel': 'cosine'}

Grid scores on development set:

-12405.497 (+/-15046.265) for {'alpha': 1, 'kernel': 'linear'}
-7650.327 (+/-8770.616) for {'alpha': 2, 'kernel': 'linear'}
-6078.911 (+/-6772.574) for {'alpha': 3, 'kernel': 'linear'}
-4756.015 (+/-5130.184) for {'alpha': 5, 'kernel': 'linear'}
-3865.324 (+/-4032.183) for {'alpha': 8, 'kernel': 'linear'}
-3091.550 (+/-3079.928) for {'alpha': 13, 'kernel': 'linear'}
-2406.822 (+/-2260.177) for {'alpha': 21, 'kernel': 'linear'}
-1789.729 (+/-1571.174) for {'alpha': 34, 'kernel': 'linear'}
-2.221 (+/-0.359) for {'alpha': 1, 'kernel': 'rbf'}
-2.231 (+/-0.362) for {'alpha': 2, 'kernel': 'rbf'}
-2.237 (+/-0.364) for {'alpha': 3, 'kernel': 'rbf'}
-2.243 (+/-0.366) for {'alpha': 5, 'kernel': 'rbf'}
-2.248 (+/-0.367) for {'alpha': 8, 'kernel': 'rbf'}
-2.252 (+/-0.368) for {'alpha': 13, 'kernel': 'rbf'}
-2.254 (+/-0.369) for {'alpha': 21, 'kernel': 'rbf'}
-2.256 (+/-0.369) for {'alp

/home/luis/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.



ValueError: continuous is not supported